In [ ]:
!pip3 install num2words

In [ ]:
import numpy as np
import nltk
import math
from num2words import num2words
from collections import Counter
nltk.download("wordnet")
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# read all data file
filename = "../files/cacm.all_processed"

with open(filename, 'r') as file:
    linesArr = file.readlines()

In [ ]:
# remove newlines
for i in range(len(linesArr)):
    linesArr[i] = linesArr[i].replace("\n", "")

In [ ]:
# create array for each document
documentsArr = []
document = []
for i in range(len(linesArr)):
    line = linesArr[i]
    if(line == "SPLITTER_TEXT"):
        if(i != 0):
            documentsArr.append(document)    
        document = []
    else:
        document.append(line)
        
if(len(documentsArr) == 3204):
    print("All documents have been successfully seperated.")

In [ ]:
documents = {}

for document in documentsArr:
    docObj = {}
    
    docID = int(document[0].split(" ")[1])
    
    title = ""
    
    indexofT = document.index(".T")
    indexofB = document.index(".B")
    
    if ".W" in document:
        abstract = ""
        indexofW = document.index(".W")
        for i in range(indexofT + 1, indexofW):
            title += " " + document[i]
        for i in range(indexofW + 1, indexofB):
            abstract += " " + document[i]
        docObj["abstract"] = abstract[1:]
        
    else:
        for i in range(indexofT + 1, indexofB):
            title += " " + document[i]
            
    docObj["title"] = title[1:]        
            
    if ".A" in document:
        authors = ""
        indexofA = document.index(".A")
        
        if ".K" in document:
            indexofK = document.index(".K")
            for i in range(indexofA + 1, indexofK):
                authors += " " + document[i]
                
        else:
            for i in range(indexofA + 1, len(document)):
                authors += " " + document[i]
            
        docObj["authors"] = authors[1:]
        
    if ".K" in document:
        keywords = ""
        indexofK = document.index(".K")
        for i in range(indexofK + 1, len(document)):
            keywords += " " + document[i]
        docObj["keywords"] = keywords[1:]
    

    
    documents[docID] = docObj

if(len(documents) == 3204):
    print("All documents have been successfully parsed.")

In [ ]:
# read queries file
filename = "../files/query.text"

with open(filename, 'r') as file:
    linesArr = file.readlines()

In [ ]:
# remove newlines
for i in range(len(linesArr)):
    linesArr[i] = linesArr[i].replace("\n", "")

In [ ]:
# create array for each query
queriesArr = []
query = []
for i in range(len(linesArr)):
    line = linesArr[i]
    if(line == " " or line == ""):
        if(i != 0):
            queriesArr.append(query)    
        query = []
    else:
        query.append(line)
        
if(len(queriesArr) == 64):
    print("All queries have been successfully seperated.")

In [ ]:
# additional parsing and organization of queries
queries = {}

for query in queriesArr:
    queryText = ""
    queryID = int(query[0].split(" ")[1])
    indexofW = query.index(".W")
    if ".A" in query:
        indexofA = query.index(".A")
        for i in range(indexofW + 1, indexofA):
            queryText += " " + query[i]
    else:
        indexofN = query.index(".N")
        for i in range(indexofW + 1, indexofN):
            queryText += " " + query[i]
    queries[queryID] = queryText[2:]
    
if(len(queries) == 64):
    print("All queries have been successfully parsed.")  

In [ ]:
# read query relations file
filename = "../files/qrels.text"

with open(filename, 'r') as file:
    linesArr = file.readlines()

In [ ]:
# create dictionary to store query relations
queryRelations = {}

for line in linesArr:
    line = line.split(" ")
    queryID = int(line[0])
    docID = int(line[1])
    
    if queryID in queryRelations:
        temp = queryRelations[queryID]
        temp.append(docID)
        queryRelations[queryID] = temp
    else:
        queryRelations[queryID] = [docID]

if(len(queryRelations) == 52):
    print("All query relations have been successfully parsed.")

In [ ]:
# read query relations file
filename = "../files/common_words"

with open(filename, 'r') as file:
    common_words = file.readlines()

if len(common_words) == 429:
    print("All common words have been successfully parsed.")

In [ ]:
# define sub-preprocessing functions
def convert_to_lowercase(text):
    return np.array_str(np.char.lower(text))

def remove_punctuation(text):
    symbols = "!\"#$%&()*+-./:;,<=>?@[\]^_`{|}~\n"
    for i in symbols:
        text = np.array_str(np.char.replace(text, i, ' '))
    return text  

def remove_apostrophe(text):
    return np.array_str(np.char.replace(text, "'", ""))

def remove_single_characters(text):
    new_text = ""
    words = text.split()
    for w in words:
        if len(w) > 1:
           new_text += " " + w
    if len(new_text) > 0 and new_text[0] == " ":
        new_text = new_text[1:]
        
    return new_text

def stem_words(text):
    stemmer = nltk.stem.PorterStemmer()
    new_text = ""
    words = text.split()
    for w in words:
        w = stemmer.stem(w) 
        new_text += " " + w
        
    if new_text[0] == " ":
        new_text = new_text[1:]
        
    return new_text

def lemmatize_words(text):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    new_text = ""
    words = text.split()
    for w in words:
        w = lemmatizer.lemmatize(w) 
        new_text += " " + w
        
    if len(new_text) > 0 and new_text[0] == " ":
        new_text = new_text[1:]
        
    return new_text

def remove_stopwords(text, choice):
    stopword_list = []
    if choice == "own":
        stopword_list = common_words
    elif choice == "nltk":
        stopword_list = nltk.corpus.stopwords.words("english")
    else:
        print("Choose 'own' or 'nltk'.")
        return text
        
    new_text = ""
    words = text.split()
    for w in words:
        if w not in stopword_list:
           new_text += " " + w
    if len(new_text) > 0 and new_text[0] == " ":
        new_text = new_text[1:]
        
    return new_text



def convert_numbers(data):
    tokens = nltk.tokenize.word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    
    new_text = np.array_str(np.char.replace(new_text, "-", " "))
                            
    return new_text


In [ ]:
# main preprocessing function
def preprocess(text):
    text = convert_to_lowercase(text)
    text = remove_punctuation(text)
    text = remove_apostrophe(text)
    text = remove_stopwords(text, "nltk")
    text = convert_numbers(text)
    text = lemmatize_words(text)
    text = remove_punctuation(text)
    text = convert_numbers(text)
    text = lemmatize_words(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text, "nltk")
    text = remove_single_characters(text)
    
    #text = remove_stopwords(text, "own")
    #text = stem_words(text)
    
    return text

# Processing

In [ ]:
processed_title = {}
processed_abstract = {}
processed_keywords = {}
processed_authors = {}


for i in range(1, len(documents)+1):
    doc = documents.get(i)
    title = doc["title"]
    abstract = ""
    authors = ""
    keywords = ""
    
    if "abstract" in doc:
        abstract = doc["abstract"]   
    if "keywords" in doc:
        keywords = doc["keywords"]  
    if "authors" in doc:
        authors = doc["authors"]
    
    processed_title[i] = nltk.tokenize.word_tokenize(preprocess(title))
    processed_abstract[i] = nltk.tokenize.word_tokenize(preprocess(abstract))
    processed_keywords[i] = nltk.tokenize.word_tokenize(preprocess(keywords))
    processed_authors[i] = nltk.tokenize.word_tokenize(preprocess(authors))
    

In [ ]:
DF = {}

for i in range(1, len(processed_title) + 1):
    
    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_abstract[i]
    
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
        tokens = processed_keywords[i]
        
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
            
        tokens = processed_authors[i]
        
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
            
for i in DF:
    DF[i] = len(DF[i])

In [ ]:
total_vocab_size = len(DF)

total_vocab = [x for x in DF]

N = len(documents)

In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

## TF-IDF for Title

In [ ]:
doc = 1

tf_idf_title = {}

for i in range(1, N + 1):
    
    tokens = processed_title[i]
    
    counter = Counter(tokens + processed_abstract[i] + processed_keywords[i] + processed_authors[i])
    words_count = len(tokens + processed_abstract[i] + processed_keywords[i] + processed_authors[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

## TF-IDF for Abstract

In [ ]:
doc = 1

tf_idf_abstract = {}

for i in range(1, N + 1):
    
    tokens = processed_abstract[i]
    
    counter = Counter(tokens + processed_title[i] + processed_keywords[i] + processed_authors[i])
    words_count = len(tokens + processed_title[i] + processed_keywords[i] + processed_authors[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf_abstract[doc, token] = tf*idf

    doc += 1

## TF-IDF for Keywords

In [ ]:
doc = 1

tf_idf_keywords = {}

for i in range(1, N + 1):
    
    tokens = processed_keywords[i]
    
    counter = Counter(tokens + processed_title[i] + processed_abstract[i] + processed_authors[i])
    words_count = len(tokens + processed_title[i] + processed_abstract[i] + processed_authors[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf_keywords[doc, token] = tf*idf

    doc += 1

## TF-IDF for Authors

In [ ]:
doc = 1

tf_idf_authors = {}

for i in range(1, N + 1):
    
    tokens = processed_authors[i]
    
    counter = Counter(tokens + processed_title[i] + processed_abstract[i] + processed_keywords[i])
    words_count = len(tokens + processed_title[i] + processed_abstract[i] + processed_keywords[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf_authors[doc, token] = tf*idf

    doc += 1

### Merging TF-IDF

In [273]:
for i in tf_idf_title:
    tf_idf_title[i] *= 0.5
    
for i in tf_idf_abstract:
    tf_idf_title[i] = tf_idf_abstract[i]
    
for i in tf_idf_keywords:
    tf_idf_title[i] = tf_idf_keywords[i]

for i in tf_idf_authors:
    tf_idf_title[i] = tf_idf_authors[i]

# Average Precision

In [ ]:
def average_precision(predictions, actual):
    counter = 0
    precisions = []
    
    for i in range(len(predictions)):
        pred = predictions[i]
        if pred in actual:
            counter += 1
            precisions.append(counter/(i+1))
            
    if counter == 0:
        return 0
    else:
        return np.mean(precisions)

## Create qrels

In [243]:
def create_qrels(filename, queryNum, docs):
    
    f = open("./output/" + filename, "a")
    for doc in docs:
        queryNumStr = str(queryNum)
        docStr = str(doc)
        
        while len(queryNumStr) < 2:
            queryNumStr = "0" + queryNumStr
            
        while len(docStr) < 4:
            docStr = "0" + docStr    
        
        line = "" + queryNumStr + " " + docStr + "\n"
        f.write(line)
    f.close()
    

## Matching Score

In [279]:
def matching_score(k, queryNum):
    query = queries[queryNum]
    preprocessed_query = preprocess(query)
    tokens = nltk.tokenize.word_tokenize(str(preprocessed_query))
    match_counter = 0
    query_weights = {}

    for key in tf_idf_title:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf_title[key]
            except:
                query_weights[key[0]] = tf_idf_title[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
    
    #create_qrels("matching_score_qrels.text", queryNum, l)
    
    show = False
    for item in l:
        if queryNum in queryRelations and item in queryRelations[queryNum]:
            show = True
            match_counter += 1
            
            
    if queryNum in queryRelations:
        recall = round(match_counter/len(queryRelations[queryNum]), 2)*100
    else:
        recall = 0
    
    #print("Q:", queryNum, "Precision:", round((match_counter/k), 2)*100, "%" , "Recall:", recall)
    
    if queryNum in queryRelations:
        avg_prec = average_precision(l, queryRelations[queryNum])
    else:
        avg_prec = 0
        
    print("Q:", queryNum, "Matching Score Average Precision:", format(avg_prec*100, ".2f"), "%", "Recall:", format(recall, ".2f"), "%")
    
    if show:
        #print("Query:", query)
        #print("Predictions:", out)
        #print("Actual:", queryRelations[queryNum])
        return avg_prec, 1
    else:
        return avg_prec, 0

In [280]:
k = 20
counter = 0
avg_precisions = []
for n in queries:
    avg_prec, cnt = matching_score(k, n)
    counter += cnt
    avg_precisions.append(avg_prec)
    
mean_avg_precision = np.mean(avg_precisions)    

#print(round((counter / len(queryRelations)) * 100, 1), "%")
print("Matching score MAP: ", round(mean_avg_precision, 2)*100, "%")
print("Matching score MAP(without empty queries): ", round((mean_avg_precision*64)/52, 2)*100, "%")

Q: 1 Matching Score Average Precision: 10.99 % Recall: 40.00 %
Q: 2 Matching Score Average Precision: 12.50 % Recall: 33.00 %
Q: 3 Matching Score Average Precision: 0.00 % Recall: 0.00 %
Q: 4 Matching Score Average Precision: 26.19 % Recall: 25.00 %
Q: 5 Matching Score Average Precision: 23.32 % Recall: 50.00 %
Q: 6 Matching Score Average Precision: 10.10 % Recall: 67.00 %
Q: 7 Matching Score Average Precision: 58.43 % Recall: 25.00 %
Q: 8 Matching Score Average Precision: 10.00 % Recall: 33.00 %
Q: 9 Matching Score Average Precision: 26.85 % Recall: 33.00 %
Q: 10 Matching Score Average Precision: 49.44 % Recall: 23.00 %
Q: 11 Matching Score Average Precision: 52.01 % Recall: 42.00 %
Q: 12 Matching Score Average Precision: 52.92 % Recall: 60.00 %
Q: 13 Matching Score Average Precision: 38.84 % Recall: 45.00 %
Q: 14 Matching Score Average Precision: 83.33 % Recall: 5.00 %
Q: 15 Matching Score Average Precision: 30.28 % Recall: 30.00 %
Q: 16 Matching Score Average Precision: 33.66 % Reca

## Cosine Similarity

In [283]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

D = np.zeros((N, total_vocab_size))
for i in tf_idf_title:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf_title[i]
    except:
        pass
    


def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q


def cosine_similarity(k, queryNum):
    query = queries[queryNum]
    preprocessed_query = preprocess(query)
    tokens = nltk.tokenize.word_tokenize(str(preprocessed_query))
    match_counter = 0
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k-1:][::-1]
    
    out = np.delete(out, 0)
    
    #create_qrels("cosine_sim_qrels.text", queryNum, out)
    
    show = False
    for item in out:
        if queryNum in queryRelations and item in queryRelations[queryNum]:
            show = True
            match_counter += 1
            
    if queryNum in queryRelations:
        recall = round(match_counter/len(queryRelations[queryNum]), 2)*100
    else:
        recall = 0
    
    #print("Q:", queryNum, "Precision:", round((match_counter/k), 2)*100, "%" , "Recall:", recall)
    
    if queryNum in queryRelations:
        avg_prec = average_precision(out, queryRelations[queryNum])
    else:
        avg_prec = 0
        
    print("Q:", queryNum, "Cosine Similarity Average Precision:", format(avg_prec*100, ".2f"), "%", "Recall:", format(recall, ".2f"), "%")
    
    if show:
        #print("Query:", query)
        #print("Predictions:", out)
        #print("Actual:", queryRelations[queryNum])
        return avg_prec, 1
    else:
        return avg_prec, 0

In [284]:
k = 20
counter = 0
avg_precisions = []
for n in queries:
    avg_prec, cnt = cosine_similarity(k, n)
    counter += cnt
    avg_precisions.append(avg_prec)
    
mean_avg_precision = np.mean(avg_precisions)    

#print(round((counter / len(queryRelations)) * 100, 1), "%")
print("Cosine similarity MAP: ", round(mean_avg_precision, 2)*100, "%")
print("Cosine similarity MAP(without empty queries): ", round((mean_avg_precision*64)/52, 2)*100, "%")

/Users/blackcherry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Q: 1 Cosine Similarity Average Precision: 19.15 % Recall: 60.00 %
Q: 2 Cosine Similarity Average Precision: 33.54 % Recall: 100.00 %
Q: 3 Cosine Similarity Average Precision: 11.11 % Recall: 17.00 %
Q: 4 Cosine Similarity Average Precision: 56.67 % Recall: 25.00 %
Q: 5 Cosine Similarity Average Precision: 20.77 % Recall: 50.00 %
Q: 6 Cosine Similarity Average Precision: 44.77 % Recall: 100.00 %
Q: 7 Cosine Similarity Average Precision: 69.76 % Recall: 29.00 %
Q: 8 Cosine Similarity Average Precision: 14.29 % Recall: 33.00 %
Q: 9 Cosine Similarity Average Precision: 18.80 % Recall: 33.00 %
Q: 10 Cosine Similarity Average Precision: 90.60 % Recall: 46.00 %
Q: 11 Cosine Similarity Average Precision: 70.70 % Recall: 47.00 %
Q: 12 Cosine Similarity Average Precision: 74.36 % Recall: 60.00 %
Q: 13 Cosine Similarity Average Precision: 76.56 % Recall: 45.00 %
Q: 14 Cosine Similarity Average Precision: 48.49 % Recall: 11.00 %
Q: 15 Cosine Similarity Average Precision: 27.22 % Recall: 40.00 %
Q: